# Test Firestore

In [4]:
# https://firebase.google.com/docs/firestore/quickstart?authuser=0&_gl=1*2w73qq*_ga*OTM3NzQyMjU5LjE2OTAzMDU0MTU.*_ga_CW55HF8NVT*MTY5MDQ2MDMyNC4yLjEuMTY5MDQ2MTA3MC4wLjAuMA..#python
# https://www.youtube.com/watch?v=b4W3YQdViTI
# https://www.youtube.com/watch?v=N0j6Fe2vAK4

import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore

# Use a service account.
cred = credentials.Certificate('/Users/vladbordei/Documents/Development/ProductExplorer/notebooks/productexplorerdata-firebase-adminsdk-ulb3d-465f23dff3.json')

app = firebase_admin.initialize_app(cred)

db = firestore.client()

In [7]:
Obj1 = {
'Name':'Tony',
'Age': 100,
'Net Worth': 10000000
}

Obj2 = {
'Name':'Mike',
'Age': 10,
'Net Worth': 100000000
}

data = [Obj1, Obj2]
data

[{'Name': 'Tony', 'Age': 100, 'Net Worth': 10000000},
 {'Name': 'Mike', 'Age': 10, 'Net Worth': 100000000}]

In [9]:
# write data to firestore
for record in data:
    print(record)
    doc_ref = db.collection("users").document(record["Name"])
    doc_ref.set(record)

{'Name': 'Tony', 'Age': 100, 'Net Worth': 10000000}
{'Name': 'Mike', 'Age': 10, 'Net Worth': 100000000}


In [15]:
# Read data
users_ref = db.collection("users")
docs = users_ref.stream()

for doc in docs:
    print(f"{doc.id} => {doc.to_dict()}")

Mike => {'Age': 10, 'Net Worth': 100000000, 'Name': 'Mike'}
Tony => {'Age': 100, 'Net Worth': 10000000, 'Name': 'Tony'}


Data Structure for handling users, payments and subscriptions

/users/
    userId1:
        name: 'John Doe'
        email: 'johndoe@example.com'
        current_package: 'basic'
        remaining_analysis: 10
        /subscriptions/
            subscriptionId1:
                package: 'premium'
                start_date: '2023-06-01'
                end_date: '2023-06-30'
                payment_status: 'paid'
            subscriptionId2:
                package: 'basic'
                start_date: '2023-07-01'
                payment_status: 'paid'
    userId2: 
        ...

/payments/
    paymentId1:
        user_id: userId1
        subscription_id: subscriptionId1
        amount: 100.00
        date: '2023-06-01'
        status: 'completed'
    paymentId2:
        ...

In this setup, each user has their own document, with a subscriptions collection where you can keep track of each package they've subscribed to, along with the start and end date of each subscription.

The current_package field in the user document will store the current package of the user. The remaining_analysis field will store the remaining analysis for the current month, which you will update every time the user uses the service.

Payments are stored separately in a payments collection. Each payment is linked to a user and a subscription.


In [ ]:
import firebase_admin
from firebase_admin import credentials, firestore

cred = credentials.Certificate('path/to/serviceAccount.json')
firebase_admin.initialize_app(cred)

db = firestore.client()

def subscribe_user_to_package(user_id, package, start_date, payment_status):
    user_ref = db.collection(u'users').document(user_id)
    subs = user_ref.collection(u'subscriptions').document()
    subs.set({
        'package': package,
        'start_date': start_date,
        'payment_status': payment_status,
    })
    user_ref.update({
        'current_package': package,
        # reset remaining_analysis based on package
        'remaining_analysis': 50 if package == 'basic' else 100,  # assuming premium users get 100 analyses
    })

def log_payment(user_id, subscription_id, amount, date, status):
    payments_ref = db.collection(u'payments').document()
    payments_ref.set({
        'user_id': user_id,
        'subscription_id': subscription_id,
        'amount': amount,
        'date': date,
        'status': status,
    })

def use_analysis(user_id):
    user_ref = db.collection(u'users').document(user_id)
    user = user_ref.get()
    remaining = user.get('remaining_analysis')
    if remaining > 0:
        user_ref.update({
            'remaining_analysis': firestore.Increment(-1),
        })
        return True  # analysis was successful
    else:
        return False  # user is out of analysis


Stripe is a service for handling online payments. It provides APIs for various programming languages, including Python. You can use Stripe to process payments for the subscription packages in your app.

Here is an overview of how you could use Stripe with your Firestore database:

When a user decides to subscribe to a package, you create a new Stripe PaymentIntent using the Stripe Python API. The PaymentIntent represents a payment that the user is about to make.

Stripe will return a client_secret for the PaymentIntent. You send this client_secret to your front end. The front end uses the client_secret to confirm the payment with Stripe.

Stripe will handle the payment confirmation process, which might involve the user's bank if necessary. Once the payment is confirmed, Stripe will send a webhook event to your server.

Your server listens for the webhook event. When it receives the event, it marks the payment as completed in your Firestore database.

Here is a basic example of how you can create a PaymentIntent using the Stripe Python API:

In [ ]:
import stripe

stripe.api_key = 'your_stripe_api_key'

def create_payment(user_id, package, amount):
    payment_intent = stripe.PaymentIntent.create(
        amount=amount,  # amount in cents
        currency='usd',
        metadata={'user_id': user_id, 'package': package},
    )
    return payment_intent.client_secret


This function creates a PaymentIntent for a given user, package, and amount. It then returns the client_secret of the PaymentIntent. You would typically call this function when a user decides to subscribe to a package.

Stripe will handle the payment confirmation process and then send a webhook event to your server. You would set up a route on your server to receive these webhook events:

In [ ]:
from flask import Flask, request
import stripe

app = Flask(__name__)

stripe.api_key = 'your_stripe_api_key'

@app.route('/webhook', methods=['POST'])
def webhook_received():
    event = stripe.Webhook.construct_event(
        request.data, request.headers.get('stripe-signature'), 'your_stripe_webhook_secret'
    )

    if event['type'] == 'payment_intent.succeeded':
        payment_intent = event['data']['object']
        user_id = payment_intent['metadata']['user_id']
        package = payment_intent['metadata']['package']
        log_payment(user_id, package, payment_intent['amount'], payment_intent['created'], 'completed')

    return '', 200


This code sets up a route /webhook that listens for POST requests. When it receives a request, it tries to construct a Stripe webhook event from the request data. If the event is a payment_intent.succeeded event, it logs the payment in your Firestore database.

Please note, for security reasons, verify the event data with Stripe's API before using the metadata in the webhook payload.

Remember to replace 'your_stripe_api_key' and 'your_stripe_webhook_secret' with your actual Stripe API key and webhook secret.